In [1]:
import json
import os
from datetime import datetime
from typing import Optional

EXPENSES_FILE = 'expenses.json'


# Initialize storage file if it doesn't exist
def init_storage():
    if not os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE, 'w') as f:
            json.dump([], f)


# Load expenses
def load_expenses():
    with open(EXPENSES_FILE, 'r') as f:
        return json.load(f)


# Save expenses
def save_expenses(expenses):
    with open(EXPENSES_FILE, 'w') as f:
        json.dump(expenses, f, indent=4)


# Add expense
def add_expense(description: str, amount: float, category: Optional[str] = None):
    if amount < 0:
        print("Amount must be positive.")
        return

    expenses = load_expenses()
    expense_id = expenses[-1]['id'] + 1 if expenses else 1
    expense = {
        "id": expense_id,
        "date": datetime.now().strftime("%Y-%m-%d"),
        "description": description,
        "amount": amount,
        "category": category if category else "General"
    }
    expenses.append(expense)
    save_expenses(expenses)
    print(f"Expense added successfully (ID: {expense_id})")


# List all expenses
def list_expenses():
    expenses = load_expenses()
    if not expenses:
        print("No expenses found.")
        return
    print(f"{'ID':<4} {'Date':<12} {'Description':<20} {'Amount':<10} {'Category'}")
    for exp in expenses:
        print(f"{exp['id']:<4} {exp['date']:<12} {exp['description']:<20} ${exp['amount']:<10.2f} {exp['category']}")


# Delete expense by ID
def delete_expense(expense_id: int):
    expenses = load_expenses()
    new_expenses = [e for e in expenses if e['id'] != expense_id]
    if len(new_expenses) == len(expenses):
        print("Expense ID not found.")
        return
    save_expenses(new_expenses)
    print(f"Expense deleted successfully (ID: {expense_id})")


# Update expense by ID
def update_expense(expense_id: int, description: Optional[str] = None, amount: Optional[float] = None, category: Optional[str] = None):
    expenses = load_expenses()
    for exp in expenses:
        if exp['id'] == expense_id:
            if description:
                exp['description'] = description
            if amount is not None:
                if amount < 0:
                    print("Amount must be positive.")
                    return
                exp['amount'] = amount
            if category:
                exp['category'] = category
            save_expenses(expenses)
            print(f"Expense updated successfully (ID: {expense_id})")
            return
    print("Expense ID not found.")


# Summary of all expenses
def summary():
    expenses = load_expenses()
    total = sum(exp['amount'] for exp in expenses)
    print(f"Total expenses: ${total:.2f}")


# Summary for a specific month
def summary_month(month: int):
    if month < 1 or month > 12:
        print("Invalid month.")
        return
    expenses = load_expenses()
    this_year = datetime.now().year
    month_expenses = [
        exp for exp in expenses if
        datetime.strptime(exp['date'], "%Y-%m-%d").month == month and
        datetime.strptime(exp['date'], "%Y-%m-%d").year == this_year
    ]
    total = sum(exp['amount'] for exp in month_expenses)
    print(f"Total expenses for {datetime(1900, month, 1).strftime('%B')}: ${total:.2f}")


# Filter expenses by category
def list_by_category(category: str):
    expenses = load_expenses()
    filtered = [e for e in expenses if e['category'].lower() == category.lower()]
    if not filtered:
        print(f"No expenses found for category: {category}")
        return
    print(f"{'ID':<4} {'Date':<12} {'Description':<20} {'Amount':<10} {'Category'}")
    for exp in filtered:
        print(f"{exp['id']:<4} {exp['date']:<12} {exp['description']:<20} ${exp['amount']:<10.2f} {exp['category']}")


# Export to CSV
def export_csv(filename="expenses_export.csv"):
    import csv
    expenses = load_expenses()
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['id', 'date', 'description', 'amount', 'category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for exp in expenses:
            writer.writerow(exp)
    print(f"Expenses exported to {filename}")


# Initialize storage on first run
init_storage()


In [2]:
# Add expenses
add_expense("Lunch", 20)
add_expense("Dinner", 10, category="Food")

# List expenses
list_expenses()

# Update an expense
update_expense(1, amount=25)

# Delete an expense
delete_expense(2)

# Show summary
summary()

# Show summary for a specific month
summary_month(7)  # e.g., July

# Filter by category
list_by_category("Food")

# Export to CSV
export_csv()


Expense added successfully (ID: 1)
Expense added successfully (ID: 2)
ID   Date         Description          Amount     Category
1    2025-07-21   Lunch                $20.00      General
2    2025-07-21   Dinner               $10.00      Food
Expense updated successfully (ID: 1)
Expense deleted successfully (ID: 2)
Total expenses: $25.00
Total expenses for July: $25.00
No expenses found for category: Food
Expenses exported to expenses_export.csv
